# ONNX Tutorial - Super Resolution Vision Model - Inference

The tutorial demonstrates loading trained model weights from BlobHub and using for inference. 

The model used in this tutorial is trained to scale up images (increase image resolution).

References: 
- BlobHub model  
  https://blobhub.io/onnx-vision-models/super-resolution
- Original tutorial  
  https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html

## Table of Contents

- [Install Dependencies](#Install-Dependencies)
- [Download Model from BlobHub](#Download-Model-from-BlobHub)
- [Run Model Inference](#Run-Model-Inference)
- [Compare Results](#Compare-Results)

## Install Dependencies

The following packages are required for this tutorial:

In [ ]:
!pip install torchvision

In [ ]:
!pip install blobhub

In [ ]:
!pip install onnx

In [ ]:
!pip install onnxruntime

In [ ]:
!pip install Pillow

In [ ]:
!pip install requests

## Download Model from BlobHub

This snippet downloads model from public BlobHub blob. Blob reference:

In [ ]:
ORG_ID = "onnx-vision-models"
BLOB_ID = "super-resolution"

Model downloading code:

In [ ]:
from blobhub.blob import Blob, Revision
from blobhub.presets.onnx import Onnx, Model       

# Find blob
blob = Blob(org_id=ORG_ID, blob_id=BLOB_ID)
revision = blob.revisions.latest()

# Initialize preset
onnx = Onnx(revision=revision)

# Download and save the model
downloaded_model = onnx.download()
assert None != downloaded_model

Downloaded model is stored locally and is accessible under:

In [ ]:
downloaded_model.path

## Run Model Inference

Check model correctness (ONNX model consistency check):

In [ ]:
import onnx

onnx_model = onnx.load(downloaded_model.path)
onnx.checker.check_model(onnx_model)

Initialize ONNX runtime:

In [ ]:
import onnxruntime

ort_session = onnxruntime.InferenceSession(downloaded_model.path)

Load input image and convert to tensor:

In [ ]:
import requests
from io import BytesIO
from PIL import Image
import torchvision.transforms as transforms

# Load the image 
input_image_url = "https://static.blobhub.io/revisions/01cba831-d693-4adf-b0c8-ded7e553c019/bridge_224x224.jpg"
response = requests.get(input_image_url)
input_image_original = Image.open(BytesIO(response.content))

# Resize image to match model input dimensions
resize = transforms.Resize([224, 224])
input_image = resize(input_image_original)

# Convert image into YCbCr color space 
img_ycbcr = input_image.convert("YCbCr")
img_in_y, img_in_cb, img_in_cr = img_ycbcr.split()

# Create tensor based on luma signal
to_tensor = transforms.ToTensor()
img_in_y = to_tensor(img_in_y)
img_in_y.unsqueeze_(0)

Run inference:

In [ ]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [ ]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img_in_y)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y_out = ort_outs[0]

Construct image object based on the model output:

In [ ]:
import numpy as np

# Convert model output into an image (just Y channel)
img_out_y = Image.fromarray(np.uint8((img_out_y_out[0] * 255.0).clip(0, 255)[0]), mode='L')

# Reconstruct the image by using model-resized luma channel and maually-resized chroma components
output_image = Image.merge(
    "YCbCr", [
        img_out_y,
        img_in_cb.resize(img_out_y.size, Image.BICUBIC),
        img_in_cr.resize(img_out_y.size, Image.BICUBIC),
    ]).convert("RGB")

## Compare Results

Original image:

In [ ]:
input_image

Resized image:

In [ ]:
output_image